# DSCI 100 Review

In [1]:
library(tidyverse)
library(tidymodels)

library(palmerpenguins)
options(repr.matrix.max.rows = 6)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

In [2]:
# Step 1: Clean data
penguins_clean <- penguins |> 
  drop_na()

# Step 2: Split data
set.seed(123)
penguin_split <- initial_split(penguins_clean, prop = 0.8)
train_data <- training(penguin_split)
test_data <- testing(penguin_split)

# Step 3: Create recipe
penguin_recipe <- recipe(species ~ bill_length_mm + bill_depth_mm + flipper_length_mm + body_mass_g, data = train_data) |> 
  step_normalize(all_predictors())

# Step 4: Specify KNN model
knn_spec <- nearest_neighbor(mode = "classification", neighbors = 5) |> 
  set_engine("kknn")

# Step 5: Create workflow
penguin_workflow <- workflow() |> 
  add_model(knn_spec) |> 
  add_recipe(penguin_recipe)

# Step 6: Fit model
knn_fit <- penguin_workflow |> 
  fit(data = train_data)

# Step 7: Predict on test data
predictions <- predict(knn_fit, test_data) |> 
  bind_cols(test_data)

# Step 8: Evaluate performance
metrics(predictions, truth = species, estimate = .pred_class)

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,multiclass,0.9850746
kap,multiclass,0.9770312
